In [2]:
import pandas as pd
import numpy as np
import re

In [ ]:
#Reading the files
#set the parameter engine = python for reading train file
#and for the testing file, submit it in without mentioning 'readonly/',  then only it is working
df = pd.read_csv(r"test.csv").set_index(['ticket_id'])
dff = pd.read_csv("train.csv", encoding='mac-roman',engine = 'python').set_index(['ticket_id'])

#Now, we have to remove rows, whose compliance values are zero
dff.dropna(axis = 0, inplace = True, subset  = ['compliance'])

#We have to consider only the rows with country value = 'usa'
dff = dff[dff['country']=='USA']

#Removing the dates as a part of data preprocessign and avoiding data leakage
train = dff[list(df.columns)].drop(['ticket_issued_date','hearing_date'], axis = 1)
test = df.drop(['ticket_issued_date','hearing_date'],axis = 1)

#forming the training Y_values
Y_train = dff['compliance']
Y_train = pd.DataFrame(Y_train, index = train.index)

#Label encoder is used in data preprocessing to convert the string data in the columns to integers or floats
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#this funciton is for identifying the columns with string valeus and then converting those strings to numerical values using 
#LabelEncoder()
for x in list(train.columns):
    le = LabelEncoder()
    if train[x].astype(str).str.contains(r'[\-A-Z]+',flags=re.IGNORECASE, regex = True).any() == True :
        train[x] = le.fit_transform(np.array(train[x].values).astype(str))
        test[x] = le.fit_transform(np.array(test[x].values).astype(str))

#for this quesiton, I am using RandomForestRegressor for finding out the prediciton values
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

#Here, I am trying out to find the features importances of each feature and select only top 6 features 
clf = RandomForestRegressor(max_depth = 10, n_estimators = 40).fit(train,Y_train)

s = pd.Series(clf.feature_importances_, index = train.columns)
s.sort_values(ascending  = False,inplace = True)

features = list(s.index[:6])
train = train[features]

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix

#Now, I am splitting the data and using GridSearchCV to to maximize the 

X_train, X_test, y_train, y_test = train_test_split(train,Y_train, random_state = 0)

grid_values = {'max_depth':[20], 'n_estimators':[50,100] }

clf = GridSearchCV(RandomForestRegressor(),param_grid = grid_values,scoring = 'roc_auc')

clf.fit(X_train, y_train.values.reshape(-1,1))

#I selected the best parameters from the gridsearcgcv and trainded the Randomforestregressor using those parameters
values = clf.best_params_
test = test[features]
final_clf = RandomForestRegressor(max_depth = values['max_depth'],n_estimators = values['n_estimators'])
final_clf.fit(train, Y_train)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_3996/1339906049.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = RandomForestRegressor(max_depth = 10, n_estimators = 40).fit(train,Y_train)
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: 

RandomForestRegressor(max_depth=20)

In [9]:
roc_auc_score(y_test, y_predict)

0.9799157419914193

In [ ]:
#finally, I made a series with values of y_predicted and ticket_id as index values

ans = pd.Series(y_scores , index  = test.index.values)
